# Offshore Wind to Hydrogen Supply Chain

## Multi-Facility - Multi-Demand Model


Importing Necessary Modules

In [ ]:
from pyomo.core import *
from pyomo.environ import *
import numpy as np
import pandas as pd
import math
import time
import matplotlib.pyplot as plt

In [ ]:
#WindTurbine Costs

#WindTurbine Costs

PCWT =  3.00 #Annualized Capital Cost of 15MW Wind Turbine (M$)
POWT =  1.05 #Annual Maintenance Cost of 15MW Wind Turbine (M$)

#Desalination Costs

PCDes =  47*10**(-6)*1.35 #Annualized Capital Cost of Desalination per kt of h2 produced (M$)
PODes =  11.48*10**(-6)*1.35 #Annual Maintenance Cost of Desalination per kt of h2 produced (M$)

#Electrolyzer Costs

PCEL =  45.32*10**(0) #Annualized Capital Cost of Electrolyzer per GW of Capacity (M$)
POEL =  22.25*10**(0) #Annual Maintenance Cost of Desalination per GW of Capacity (M$)

#Liquefaction Costs

PCLiq =  1.5*627 #Annualized Capital Cost of Liquefaction per Capacity in kilotonnes per hour (M$) 8.148177*10*(3)
POLiq =  5*10**(-2)*PCLiq #Annual Maintenance Cost of Desalination per Capacity in kilotonnes per hour (M$)

#Storage Costs

PCStore =  3.667*10**(-1) #Annualized Capital Cost of Storage (M$) per kton
POStore =  9.0*10**(-2) #Annual Maintenance Cost of Storage (M$) per kton

#Ship Parameters

Shiptype = [0,1,2]
Shipsize = [1, 10, 14] #Ship capacity in kt of H2
Shipcost = [ 14256*10**(-6), 41930*10**(-6),33544*10**(-6)] #M$ per ship
transportcost = [60000*10**(-6),600000*10**(-6),840000*10**(-6)] #M$ per trip

# Floating Platform 

FP_Cost = (500/9.8)*(0.14)  #Annualized Capital Cost of Floating Platform (M$) per GW of Electrolyzer Capacity


In [ ]:
def Optimal_Design(hub,states,buoys,Factories,Demands,DemandValue,RTTFile):
    
    print('Start of New Case For:')
    
    print('Hub Name:',hub)
    print('List of States:',states)
    print('List of Buoys:',buoys)
    print('List of Factories:',Factories)
    print('List of Demands:',Demands)
    print('List of DemandValues:',DemandValue)
    print('Name of the RTT File:', RTTFile)

    filename = str(RTTFile)

    RTT = np.array(pd.read_excel(filename,header=None))

    CF =[]

    for buoy in buoys:
        CFi = pd.read_csv('CF_Data/'+str(buoy)+'.csv')
        CFii = (CFi['CF']).values.tolist()
        CF.append(CFii)
    
    Times = list(np.arange(0,365))
    
    print('------------------')
    
    ##variable Definitions
    model = ConcreteModel()

    model.selector = Var(Factories,initialize=0,within=Binary) #Facility Selection

    model.NWT = Var(Factories,initialize=0,within=Integers,bounds=(0,100000)) #No of Wind Turbines to be installed
    model.CDes = Var(Factories,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Desalination Capacity to be installed in H2 kt/hr
    model.CEL = Var(Factories,initialize=11,within=NonNegativeReals,bounds=(0,10**6)) #Electrolyzer Capacity to be installed in MW

    #0.0001 as previous lower limit for cliq,cstrore
    model.Cliq = Var(Factories,initialize=1,within=NonNegativeReals,bounds=(0,1000)) #Liquifaction Capacity to be installed in kTonnes per hr
    model.Cstore = Var(Factories,initialize=1,within=NonNegativeReals,bounds=(0,10**3)) #LH2 Storage Capacity to be installed in kTonnes


    model.NS = Var(Factories,Shiptype,initialize=1,within=Integers,bounds=(0,365)) #No of Ships of each type to be purchased
    model.NT = Var(Factories,Demands,Shiptype,initialize=1,within=Integers,bounds=(0,10**6)) #No of Trips of each type purchased
    model.s = Var(Factories,Demands,Shiptype,Times,initialize=1,within=Integers,bounds = (0,365)) #No of shipments for each shiptype

    #PowerFlow Variables
    model.EWT = Var(Factories,Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power generated by Wind Turbines in each t
    model.EDes = Var(Factories,Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power consumed by Desalination in each t
    model.EEL = Var(Factories,Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power consumed by Electrolysis in each t
    model.Eliq = Var(Factories,Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power consumed by Liquefaction in each t


    #Hydrogen Flow/Level Variables
    model.Lstore = Var(Factories,Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #LH2 Level in the tank in each t in ktonnes
    model.F = Var(Factories,Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #H2 produced in each t in ktonnes/hr

    ## Piecewise Linear Approximations
    Aliq = []
    Bliq = []
    Bstore = []
    Astore = []

    #for i in range(len(Aliq)):
    #    Bliq.append(Aliq[i]**0.6791)

    #Astore = list(np.arange(0,1100,500))  
    #for i in range(len(Astore)):
    #    Bstore.append(Astore[i]**0.6733)
        
    XUL = 1100
    X_width = 500

    model.C2liq = Var(Factories,initialize=1,within=NonNegativeReals,bounds=(0.0,XUL**0.6791))
    model.C2store = Var(Factories, initialize=1,within=NonNegativeReals,bounds=(0.0,XUL**0.6733))

    for f in Factories:
        Aliq.append(list(np.arange(0,XUL,X_width)))
        Bliq.append([])
        Bstore.append([])
        for i in range(len(Aliq[f])):
            Bliq[f].append(Aliq[f][i]**0.6791)

        Astore.append(list(np.arange(0,XUL,X_width)))
        for i in range(len(Astore[f])):
            Bstore[f].append(Astore[f][i]**0.6733)

        nameL = "constraint_LIQ"+str(f)
        model.add_component(nameL,Piecewise(model.C2liq[f],model.Cliq[f],
                            pw_pts=Aliq[f],
                            pw_constr_type='EQ',
                            f_rule=Bliq[f],
                            pw_repn='SOS2'))
        nameS = "constraint_STO"+str(f)
        model.add_component(nameS,Piecewise(model.C2store[f],model.Cstore[f],
                            pw_pts=Astore[f],
                            pw_constr_type='EQ',
                            f_rule=Bstore[f],
                            pw_repn='SOS2'))
        
    ## Piecewise_Reapproximation
    def Piecewise_Reapproximate():
        for f in Factories:
            Aliq[f].insert(0,model.Cliq[f]())
            Aliq[f].sort()
            Bliq[f].insert(Aliq[f].index(model.Cliq[f]()),model.Cliq[f]()**0.6791)
            
            Astore[f].insert(0,model.Cstore[f]())
            Astore[f].sort()
            Bstore[f].insert(Astore[f].index(model.Cstore[f]()),model.Cstore[f]()**0.673)
            
            
            nameL = "constraint_LIQ"+str(f)
            model.del_component(nameL)
            model.add_component(nameL,Piecewise(model.C2liq[f],model.Cliq[f],
                            pw_pts=Aliq[f],
                            pw_constr_type='EQ',
                            f_rule=Bliq[f],
                            pw_repn='SOS2'))
            
            nameS = "constraint_STO"+str(f)
            model.del_component(nameS)
            model.add_component(nameS,Piecewise(model.C2store[f],model.Cstore[f],
                            pw_pts=Astore[f],
                            pw_constr_type='EQ',
                            f_rule=Bstore[f],
                            pw_repn='SOS2'))
    ## OOBJ
    def oobj():
        val = (sum(
            0
            +(PCWT + POWT)*model.NWT[f]() #Capex & Opex Costs of Wind Turbines
            +(PCDes + PODes)*model.CDes[f]() #Capex & Opex Costs of Desalination
            +(PCEL + POEL)*model.CEL[f]() #Capex & Opex Costs of Electrolyzer
            +(PCLiq*model.Cliq[f]()**0.6791 + POLiq*model.Cliq[f]()**0.6791) #Capex & Opex Costs of Liquefaction
            +(PCStore*model.Cstore[f]()**0.673 + POStore*model.C2store[f]()**0.673) #Capex & Opex Costs of storage
            +sum(model.NS[f,i]()*Shipcost[i] for i in Shiptype) #Capex Costs of Ships
            +sum(model.NT[f,d,i]()*transportcost[i]*0.5*RTT[f,d] for d in Demands for i in Shiptype) #Transportation Costs of Ships
            +FP_Cost*model.CEL[f]() #Capex Costs of Floating Platform
            for f in Factories)
            )
        return val
    
    ## Objective Function 
    def objrule(model):
        
        
        return(sum(
            0
            +(PCWT + POWT)*model.NWT[f] #Capex & Opex Costs of Wind Turbines
            +(PCDes + PODes)*model.CDes[f] #Capex & Opex Costs of Desalination
            +(PCEL + POEL)*model.CEL[f] #Capex & Opex Costs of Electrolyzer
            +(PCLiq*model.C2liq[f] + POLiq*model.C2liq[f]) #Capex & Opex Costs of Liquefaction
            +(PCStore*model.C2store[f] + POStore*model.C2store[f]) #Capex & Opex Costs of storage
            +sum(model.NS[f,i]*Shipcost[i] for i in Shiptype) #Capex Costs of Ships
            +sum(model.NT[f,d,i]*transportcost[i]*0.5*RTT[f,d] for d in Demands for i in Shiptype) #Transportation Costs of Ships
            +FP_Cost*model.CEL[f] #Capex Costs of Floating Platform
            for f in Factories)
            )

    model.obj = Objective(rule=objrule, sense=minimize)


    ## Constraints

    ## Facility Selection
    BigM = 99999
    model.FacilitySelection = ConstraintList()
    for f in Factories:
        model.FacilitySelection.add(model.NWT[f]<=BigM*model.selector[f])
            
        model.FacilitySelection.add(model.CDes[f]<=BigM*model.selector[f])
        model.FacilitySelection.add(model.Cliq[f]<=BigM*model.selector[f])
        model.FacilitySelection.add(model.CEL[f]<=BigM*model.selector[f])
        model.FacilitySelection.add(model.Cstore[f]<=BigM*model.selector[f])
        for i in Shiptype:
            model.FacilitySelection.add(model.NS[f,i]<=BigM*model.selector[f])
        model.FacilitySelection.add(sum(model.NT[f,d,i] for d in Demands for i in Shiptype) >=model.selector[f])

    model.FacilitySelection.add(sum(model.selector[f] for f in Factories) == 1)

    ## Energy Balance Constraints
    model.energybalance = ConstraintList()
    for f in Factories:
        for t in Times:
            model.energybalance.add(model.EWT[f,t] == model.EDes[f,t] + model.EEL[f,t] + model.Eliq[f,t])

    model.windpower = ConstraintList()
    for f in Factories:
        for t in Times:
            model.windpower.add(model.EWT[f,t] <= CF[f][t]*0.015*24*model.NWT[f])

    model.desalination = ConstraintList()
    for f in Factories:    
        for t in Times:
            model.desalination.add(model.EDes[f,t] == 52.5*10**(-3)*model.F[f,t]) #0.0494*10**(-6)

    model.electrolysis = ConstraintList()
    for f in Factories:
        for t in Times:
            model.electrolysis.add(model.EEL[f,t] == model.F[f,t]*50*10**(0)) #check

    model.liquefaction = ConstraintList()
    for f in Factories:
        for t in Times:
            model.liquefaction.add(model.Eliq[f,t] == (model.F[f,t])*33.3*10**(0)/(3)) # Check

    ## Capacity Constraints
    model.c1 = ConstraintList()
    for f in Factories:
        for t in Times:
            model.c1.add(model.F[f,t]/24 <= model.CDes[f]) #Desalination Capacity
            

    model.c2 = ConstraintList()
    for f in Factories:
        for t in Times:
            model.c2.add(model.EEL[f,t]/24 <= model.CEL[f]) #Electrolyzer Capacity
            
        
    model.c3 = ConstraintList()
    for f in Factories:
        for t in Times:
            model.c3.add(model.F[f,t]/24 +0.015*10**(-2)*model.Lstore[f,t] <=model.Cliq[f]) #liquefaction Capacity
            
    model.c4 = ConstraintList()
    for f in Factories:
        for t in Times:
            model.c4.add(model.Lstore[f,t]<=model.Cstore[f])

    ## Transport & Inventory Constraints

    model.mb = ConstraintList()
    for f in Factories:
        for t in Times:
            if t == 0:
                model.mb.add(model.Lstore[f,t] == (model.Lstore[f,364]+model.F[f,t]-sum(model.s[f,d,i,t]*Shipsize[i] for d in Demands for i in Shiptype)))
            else:
                model.mb.add(model.Lstore[f,t] == (model.Lstore[f,t-1]+model.F[f,t]-sum(model.s[f,d,i,t]*Shipsize[i] for d in Demands for i in Shiptype)))
            
    model.sc = ConstraintList()
    for f in Factories:
        for i in Shiptype:
            for t in Times:
                model.sc.add(sum(model.s[f,d,i,t2] for d in Demands for t2 in list(np.arange(t,t+RTT[(f,d)])) if t+RTT[f,d]<=Times[-1] )<=model.NS[f,i])
                
    model.th = ConstraintList()
    for f in Factories:
        for t in Times:
            if t == 0 :
                model.th.add(sum(model.s[f,d,i,t]*Shipsize[i] for d in Demands for i in Shiptype)<=model.Lstore[f,364])
            else:
                model.th.add(sum(model.s[f,d,i,t]*Shipsize[i] for d in Demands for i in Shiptype)<=model.Lstore[f,t-1])

    model.other = ConstraintList()
    for f in Factories:
        for d in Demands:
            for i in Shiptype:
                model.other.add(model.NT[f,d,i] == sum(model.s[f,d,i,t] for t in Times))

    model.other2 = ConstraintList()
    for f in Factories:
        for i in Shiptype:
            model.other2.add(sum(model.NT[f,d,i]*(RTT[f,d]) for d in Demands) <=365*model.NS[f,i])

    ## Demand Constraints
    model.dem = ConstraintList()
    for d in Demands:
        model.dem.add(sum(model.NT[f,d,i]*Shipsize[i] for f in Factories for i in Shiptype)>= DemandValue[d])

    ## define warmstart
    def warmstarter():
        var_dict = {}
        for var in model.component_data_objects(Var, active=True, descend_into=True):
            try:
                var_dict[str(var)] =value(var); #check if the semi-colon helps
            except ValueError:   
                pass
        return var_dict

    ## Solve
    tic = time.perf_counter()

    q1=[]
    ub1=[]
    lb1=[]
    gap1=[]
    
    solver = SolverFactory('gurobi')
    solver.options['mipgap'] = 0.01
    solver.options['MIPFocus'] = 3
    #solver.options['Heuristics'] = 1
    solver.options['cuts']=3

    solver.options['ScaleFlag']=2

    solver.solve(model,tee=True)

    model.display()

    LB = model.obj()
    UB = oobj()
    Gap = 1-(abs(LB)/abs(UB))

    print("LB=",LB)
    print("UB=",UB)
    print("Gap=",Gap)
    #Change Start                        
    #q1.append(q)
    ub1.append(UB)
    lb1.append(LB)
    gap1.append(Gap)
    #Change End

    while Gap>0.001:
    
        Piecewise_Reapproximate()
        
        model.del_component('model.obj')
        model.obj = Objective(rule=objrule, sense=minimize)
        
        mip_start = warmstarter()
        solver.solve(model,warmstart=mip_start,tee=True)
        #solver.solve(model,tee=True)
        
        LB = model.obj()
        UB = oobj()
        Gap = 1-(abs(LB)/abs(UB))
        print("LB=",LB)
        print("UB=",UB)
        print("Gap=",Gap)
    #Change Start                        
        #q1.append(q)
        ub1.append(UB)
        lb1.append(LB)
        gap1.append(Gap)
    #Change End

    toc = time.perf_counter()
    print(f"Completed in {toc - tic:0.4f} seconds")

    # Loop through each variable in the model
    full_model_data = {'Model':'Northeast'}
    
    for var in model.component_data_objects(Var, active=True):
        full_model_data[var.name] = var.value
    
    
    def common_costs_Eval(d):
        val = sum((0
                +(PCWT + POWT)*model.NWT[f]() #Capex & Opex Costs of Wind Turbines
                +(PCDes + PODes)*model.CDes[f]() #Capex & Opex Costs of Desalination
                +(PCEL + POEL)*model.CEL[f]() #Capex & Opex Costs of Electrolyzer
                +(PCLiq*model.Cliq[f]()**0.6791 + POLiq*model.Cliq[f]()**0.6791) #Capex & Opex Costs of Liquefaction
                +(PCStore*model.Cstore[f]()**0.673 + POStore*model.C2store[f]()**0.673) #Capex & Opex Costs of storage
                +sum(model.NS[f,i]()*Shipcost[i] for i in Shiptype) #Capex Costs of Ships
                +FP_Cost*model.CEL[f]() #Capex & Opex Costs of Floating Platform
                ) for f in Factories)
        return val

    def sp_costs_Eval(d):
        val = sum(model.NT[f,d,i]()*transportcost[i]*0.5*RTT[f,d] for f in Factories for i in Shiptype )
        return val

    state_results = pd.DataFrame()

    for d in Demands:
        model_data = {'Model': hub}
        model_data['State'] = states[d]
        model_data['Objective'] = model.obj()
        

        model_data['Cost of Wind Turbines'] = sum((PCWT + POWT)*model.NWT[f]() for f in Factories)
        model_data['Cost of Desalination'] = sum((PCDes + PODes)*model.CDes[f]() for f in Factories)
        model_data['Cost of Electrolyzer'] = sum((PCEL + POEL)*model.CEL[f]() for f in Factories)
        model_data['Cost of Liquefaction'] = sum((PCLiq*model.Cliq[f]()**0.6791 + POLiq*model.Cliq[f]()**0.6791) for f in Factories)
        model_data['Cost of Storage'] = sum((PCStore*model.Cstore[f]()**0.673 + POStore*model.C2store[f]()**0.673) for f in Factories)
        model_data['Cost of Ships'] = sum(model.NS[f,i]()*Shipcost[i] for f in Factories for i in Shiptype)
        model_data['cost of floating platform'] = FP_Cost*sum(model.CEL[f]() for f in Factories)

        model_data['Common Costs'] = common_costs_Eval(d)
        model_data['Shipping Costs'] = sp_costs_Eval(d)
        model_data['H2 Shipped'] = sum(model.NT[f,d,i]()*Shipsize[i] for f in Factories for i in Shiptype)
        model_data['H2 Produced'] = sum(model.F[f,t]() for f in Factories for t in Times)
        model_data['Common_LCOH'] = common_costs_Eval(d)/sum(model.F[f,t]() for f in Factories for t in Times)
        model_data['Shipping_LCOH'] = sp_costs_Eval(d)/sum(model.F[f,t]() for f in Factories for t in Times)
        model_data['Total_LCOH'] = model_data['Common_LCOH'] + model_data['Shipping_LCOH']

        model_data['Energy Generated'] = sum(model.EWT[f,t]() for f in Factories for t in Times)
        model_data['Energy for Desalination'] = sum(model.EDes[f,t]() for f in Factories for t in Times)
        model_data['Energy for Electrolysis'] = sum(model.EEL[f,t]() for f in Factories for t in Times)
        model_data['Energy for Liquefaction'] = sum(model.Eliq[f,t]() for f in Factories for t in Times)
        

        model_data['GHG of Wind Turbines'] = sum(model.NWT[f]() for f in Factories)*15/11*1.65e7
        model_data['GHG of Desalination'] = model_data['H2 Produced']*(1000/2*18*20)*(2.25+4.85E-1)
        model_data['GHG of Electrolyzer'] = sum(model.CEL[f]() for f in Factories)*1000*5.85e4
        model_data['GHG of Liquefaction'] = sum(model.Cliq[f]() for f in Factories)*1000*24*1.15e5
        model_data['GHG of Storage'] = sum(model.Cstore[f]() for f in Factories)*1000*2.04e3
          
        model_data['GHG of Transport'] = sum(model.NT[f,d,i]()*model.NT[f,d,i]()*Shipsize[i] for f in Factories for i in Shiptype)*(1000*RTT[f,d]*100*20)*(1.90e-2)
     
        state_results = state_results.append(model_data, ignore_index=True)

    return full_model_data, state_results

In [ ]:
df = pd.read_excel('Hubwise_Casestudy_guide.xlsx')

#Get Hubs
hubs = df['Hub'].tolist()

#get states
states = df['states'].tolist()
#convert comma separated string to list
states = [i.split(',') for i in states]

#Get Buoys
buoys = df['buoys'].tolist()
#convert comma separated string to list
buoys = [i.split(',') for i in buoys]

#Get Factories
Factories = df['Factories'].tolist()
#convert comma separated string to list
Factories = [i.split(',') for i in Factories]

#convert str numbers to int
for i in range(len(Factories)):
    Factories[i] = [int(j) for j in Factories[i]]

#Get Demands
Demands = df['Demands'].tolist()
#convert comma separated string to list
Demands = [i.split(',') for i in Demands]

#convert str numbers to int
for i in range(len(Demands)):
    Demands[i] = [int(j) for j in Demands[i]]

#Get DemandValues
DemandValue = df['DemandValue'].tolist()
#convert comma separated string to list
DemandValue = [i.split(',') for i in DemandValue]

#convert str numbers to float
for i in range(len(DemandValue)):
    DemandValue[i] = [float(j) for j in DemandValue[i]]

#Get RTT filenames
RTTFile = df['RTTFile'].tolist()

full_states_results = pd.DataFrame()

for i in range(len(hubs)):
    full_model_data, state_results = Optimal_Design(hubs[i],states[i],buoys[i],Factories[i],Demands[i],DemandValue[i],RTTFile[i])
    
    full_model_data = pd.DataFrame.from_dict(full_model_data, orient='index')
    full_model_data = full_model_data.transpose()
    full_model_data.to_csv('Fullmodel_results_Hubwise_LH2' +str(hubs[i])+'hub.csv')
    
    full_states_results = full_states_results.append(state_results, ignore_index=True)

full_states_results.to_excel('Hubwise_LH2_final_results.xlsx')
